In [3]:
import re
import random
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

from keras.layers import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.models import Sequential

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
base_path = "brazilian-songs-lyrics/"
base_filename = "letras_mus_br_"
axe = "axe.csv"
bossa_nova = "bossa-nova.csv"
forro = "forro.csv"
funk = "funk.csv"
gospel = "gospel.csv"
mpb = "mpb.csv"
pagode = "pagode.csv"
samba = "samba.csv"
sertanejo = "sertanejo.csv"

In [17]:
# letras = [axe, bossa_nova, forro, funk, gospel, mpb, pagode, samba, sertanejo]
letras = [sertanejo]

#### Textos

In [26]:
def text_basic(texto):
    texto = re.sub(r'[\s\n]+', " ", texto)
    return texto.lower()

In [27]:
def get_text(num_samples = 0):
    textos = []
    for letra in letras:
        file = pd.read_csv(base_path + base_filename + letra)
        if(num_samples > 0):
            samples = random.sample(list(file.letras), num_samples)
        else: 
            samples = file.letras
        textos.append(samples)
        print(letra, len(samples))
    textos = np.array(textos).ravel()
    print("total", len(textos))
    textos = " ".join(textos).strip()
    return text_basic(textos)

#### Variáveis

In [14]:
SEQLEN_CHAR, SEQLEN_WORD, STEP = 15, 5, 1
BATCH_SIZE = 128
HIDDEN_SIZE = 128
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS = 200

#### Create Model

In [15]:
def create_model(sequence, size):
    model = Sequential()
    model.add(
        LSTM(  
            HIDDEN_SIZE,
            return_sequences=False,
            input_shape=(sequence, size),
            unroll=True
        )
    )
    model.add(Dense(size))
    model.add(Activation("softmax"))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Treinamento por char

In [28]:
textos = get_text()
textos[:1000]

sertanejo.csv 1000
total 1000


'já doeu. mas hoje não dói mais. tanto fiz. que agora tanto faz. o nosso amor calejou. apanhou, apanhou que cansou. na minha cama cê fez tanta falta. que o meu coração te expulsou. não tem mais eu e você. tá facin de entender. você me deu aula de como aprender te esquecer. foi, mas não é mais a minha notificação preferida. já foi, mas não é mais a número um da minha vida. sinto em te dizer. mas eu já superei você. o nosso amor calejou. apanhou, apanhou que cansou. na minha cama cê fez tanta falta. que o meu coração te expulsou. não tem mais eu e você. tá facin de entender. você me deu aula de como aprender te esquecer. foi, mas não é mais a minha notificação preferida. já foi, mas não é mais a número um da minha vida. sinto em te dizer. mas eu já superei você. foi, mas não é mais a minha notificação preferida. já foi, mas não é mais a número um da minha vida. sinto em te dizer. mas eu já superei você. já doeu. mas hoje não dói mais. no começo, eu entendia. mas era só cama, não tinha am

In [29]:
input_chars, label_chars = [], []

chars = set([c for c in textos])
nb_chars = len(chars)

# Cria um mapeamento de letras para números e vice-versa
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# Converte os dados em uma série de subsequencias de tamanho 10
for i in range(0, len(textos) - SEQLEN_CHAR, STEP):
    input_chars.append(textos[i: i + SEQLEN_CHAR])
    label_chars.append(textos[i + SEQLEN_CHAR])
    
# Cria o vetor one-hot encoding das sequencias de entradas (X) e o próximo caracter (y)
X = np.zeros((len(input_chars), SEQLEN_CHAR, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [31]:
# Divide o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=42)

model = create_model(SEQLEN_CHAR, nb_chars)
EPOCS = 100

for iteration in range(EPOCS):
    print("=" * 50)
    print("Iteração #: {}".format(iteration))
    model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

Iteração #: 0
Epoch 1/1
656249/656249 [==============================] - 55s 84us/step - loss: 0.0428 - accuracy: 0.9867
Iteração #: 1
Epoch 1/1
656249/656249 [==============================] - 53s 81us/step - loss: 0.0364 - accuracy: 0.9881
Iteração #: 2
Epoch 1/1
656249/656249 [==============================] - 53s 81us/step - loss: 0.0336 - accuracy: 0.9890
Iteração #: 3
Epoch 1/1
656249/656249 [==============================] - 57s 87us/step - loss: 0.0320 - accuracy: 0.9895
Iteração #: 4
Epoch 1/1
656249/656249 [==============================] - 54s 82us/step - loss: 0.0309 - accuracy: 0.9898
Iteração #: 5
Epoch 1/1
656249/656249 [==============================] - 53s 80us/step - loss: 0.0301 - accuracy: 0.9901
Iteração #: 6
Epoch 1/1
656249/656249 [==============================] - 53s 81us/step - loss: 0.0295 - accuracy: 0.9903
Iteração #: 7
Epoch 1/1
656249/656249 [==============================] - 58s 88us/step - loss: 0.0290 - accuracy: 0.9904
Iteração #: 8
Epoch 1/1
656249/6

656249/656249 [==============================] - 59s 91us/step - loss: 0.0239 - accuracy: 0.9922
Iteração #: 47
Epoch 1/1
656249/656249 [==============================] - 58s 88us/step - loss: 0.0239 - accuracy: 0.9922
Iteração #: 48
Epoch 1/1
656249/656249 [==============================] - 63s 96us/step - loss: 0.0239 - accuracy: 0.9922
Iteração #: 49
Epoch 1/1
656249/656249 [==============================] - 64s 97us/step - loss: 0.0238 - accuracy: 0.9922
Iteração #: 50
Epoch 1/1
656249/656249 [==============================] - 59s 89us/step - loss: 0.0238 - accuracy: 0.9922
Iteração #: 51
Epoch 1/1
656249/656249 [==============================] - 59s 90us/step - loss: 0.0238 - accuracy: 0.9922
Iteração #: 52
Epoch 1/1
656249/656249 [==============================] - 59s 90us/step - loss: 0.0238 - accuracy: 0.9922
Iteração #: 53
Epoch 1/1
656249/656249 [==============================] - 57s 86us/step - loss: 0.0237 - accuracy: 0.9922
Iteração #: 54
Epoch 1/1
656249/656249 [=========

656249/656249 [==============================] - 60s 92us/step - loss: 0.0232 - accuracy: 0.9924
Iteração #: 94
Epoch 1/1
656249/656249 [==============================] - 59s 90us/step - loss: 0.0232 - accuracy: 0.9924
Iteração #: 95
Epoch 1/1
656249/656249 [==============================] - 57s 87us/step - loss: 0.0232 - accuracy: 0.9924
Iteração #: 96
Epoch 1/1
656249/656249 [==============================] - 58s 89us/step - loss: 0.0232 - accuracy: 0.9924
Iteração #: 97
Epoch 1/1
656249/656249 [==============================] - 61s 93us/step - loss: 0.0232 - accuracy: 0.9924
Iteração #: 98
Epoch 1/1
656249/656249 [==============================] - 64s 97us/step - loss: 0.0231 - accuracy: 0.9924
Iteração #: 99
Epoch 1/1
656249/656249 [==============================] - 63s 96us/step - loss: 0.0231 - accuracy: 0.9924


In [32]:
# Avalia o modelo
loss, accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=0)
print(model.metrics_names)
print("Accuracy: ", accuracy)
print("Loss: ", loss)

['loss', 'accuracy']
Accuracy:  0.9907665848731995
Loss:  0.02929864006459817


In [33]:
def completar_frase_caractere(frase_inicial):
    frase = frase_inicial
    for i in range(NUM_PREDS):
        Xtest = np.zeros((1, SEQLEN_CHAR, nb_chars))
        for j, ch in enumerate(frase[-15:]):
            Xtest[0, j, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]

        # Exibe o próximo caracter previsto
        print(ypred, end="")
        frase = frase[1:] + ypred

### Resultados treinamento caractere

In [34]:
frases = ["hoje eu acordei com vontade ",
         "hoje ela vai dançar e ",
         "preciso de você e ",
         "se a gente conseguir ficar ",
         "pensei em você quando ",
         "meu coração se sente "]

for index, frase in enumerate(frases):
    print("Frase %i: %s" % (index, frase), end="")
    completar_frase_caractere(frase)
    print("\n")

Frase 0: hoje eu acordei com vontade de viver. que eu tô dante e vou manhando por aí. e o seu cantinho perdidos. mas eu vou te esquecer, mas eu vou te esquecer, mas eu vou te esquecer, mas eu vou te esquecer, mas eu vou te esquecer, mas 

Frase 1: hoje ela vai dançar e a gente não começou a sua vida. e a gente fica com a vida e não tem mais cala. se eu te amar assim. eu não vou deixar de amor. e a sua roupa é pra te ver. se eu te amais. nem sei que se algum amor. eu

Frase 2: preciso de você e contar. e aí, a sale da sempre vem. por causa do que a gente faz isso pra ele. sou seu amor por você. eu não vou deixar de amor. e a sua roupa é pra te ver. se eu te amais. nem sei que se algum amor. 

Frase 3: se a gente conseguir ficar na sua boca. e eu acho que se acha que a gente tá bem sem ser a sua vida. eu sei que você está na minha vida. e eu não vou te amar. e a gente não começou a sua vida. e a gente fica com a vida e não te

Frase 4: pensei em você quando a gente fica ainda está na uma

# Treinamento por palavra

In [55]:
textos = get_text(10)
textos[:1000]

axe.csv 10
bossa-nova.csv 10
forro.csv 10
funk.csv 10
gospel.csv 10
mpb.csv 10
pagode.csv 10
samba.csv 10
sertanejo.csv 10
total 90


'não esta sendo fácil é difícil não chorar quando vejo nossas fotos no meu celular não é fácil apagar as lembranças eu só penso em nós por favor não desligue agora preciso ouvir sua voz estou só vem me ver tô sofrendo querendo seus beijos só depende de você não depende de mim te perdoo porque eu te amo e não vivo sem você amor eu te amo eu te amo e não vivo sem você só depende de você não depende de mim te perdoo porque eu te amo e não vivo sem você no te pares frente a mí con esa mirada tan hiriente puedo entender estrechez de mente soportar la falta de experiencia pero no voy a aguantar estrechez de corazón no vuelvas a hablar así no rebajes estas relaciones si vivimos de cariño y besos no me digas de odios y traiciones cuántas cosas se dirán en la guerra del amor las palabras son cuchillas cuándo las manejan orgullos y pasiones estás llorando y no haces nada por comprender a nadie excepto a ti oye no voy a aguantar tú no puedes demostrar oye no voy a aguantar estrechez de corazón no

In [64]:
input_words, label_words = [], []

word_list = word_tokenize(textos)
words = set([w for w in word_list])
nb_words = len(words)

# Cria um mapeamento de palavras para números e vice-versa
word2index = {w: i for i, w in enumerate(words)}
index2word = {i: w for i, w in enumerate(words)}

# Converte os dados em uma série de subsequencias de tamanho 10
for i in range(0, len(word_list) - SEQLEN_WORD, STEP):
    input_words.append(word_list[i: i + SEQLEN_WORD])
    label_words.append(word_list[i + SEQLEN_WORD])
    
# Cria o vetor one-hot encoding das sequencias de entradas (X) e o próximo caracter (y)
X = np.zeros((len(input_words), SEQLEN_WORD, nb_words), dtype=np.bool)
y = np.zeros((len(input_words), nb_words), dtype=np.bool)
for i, input_word in enumerate(input_words):
    for j, w in enumerate(input_word):
        X[i, j, word2index[w]] = 1
    y[i, word2index[label_words[i]]] = 1

In [77]:
# Divide o dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=42)

model = create_model(SEQLEN_WORD, nb_words)
EPOCS = 100

for iteration in range(EPOCS):
    print("=" * 50)
    print("Iteração #: {}".format(iteration))
    model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

Iteração #: 0
Epoch 1/1
12475/12475 [==============================] - 29s 2ms/step - loss: 0.0031 - accuracy: 0.9997
Iteração #: 1
Epoch 1/1
12475/12475 [==============================] - 28s 2ms/step - loss: 0.0028 - accuracy: 0.9997
Iteração #: 2
Epoch 1/1
12475/12475 [==============================] - 27s 2ms/step - loss: 0.0026 - accuracy: 0.9997
Iteração #: 3
Epoch 1/1
12475/12475 [==============================] - 28s 2ms/step - loss: 0.0025 - accuracy: 0.9997
Iteração #: 4
Epoch 1/1
12475/12475 [==============================] - 28s 2ms/step - loss: 0.0025 - accuracy: 0.9997
Iteração #: 5
Epoch 1/1
12475/12475 [==============================] - 29s 2ms/step - loss: 0.0025 - accuracy: 0.9997
Iteração #: 6
Epoch 1/1
12475/12475 [==============================] - 30s 2ms/step - loss: 0.0025 - accuracy: 0.9997
Iteração #: 7
Epoch 1/1
12475/12475 [==============================] - 30s 2ms/step - loss: 0.0025 - accuracy: 0.9997
Iteração #: 8
Epoch 1/1
12475/12475 [===================

12475/12475 [==============================] - 28s 2ms/step - loss: 8.1750e-04 - accuracy: 0.9998
Iteração #: 49
Epoch 1/1
12475/12475 [==============================] - 28s 2ms/step - loss: 7.8379e-04 - accuracy: 0.9998
Iteração #: 50
Epoch 1/1
12475/12475 [==============================] - 29s 2ms/step - loss: 7.5583e-04 - accuracy: 0.9998
Iteração #: 51
Epoch 1/1
12475/12475 [==============================] - 29s 2ms/step - loss: 7.2980e-04 - accuracy: 0.9998
Iteração #: 52
Epoch 1/1
12475/12475 [==============================] - 29s 2ms/step - loss: 7.0345e-04 - accuracy: 0.9998
Iteração #: 53
Epoch 1/1
12475/12475 [==============================] - 27s 2ms/step - loss: 6.7505e-04 - accuracy: 0.9998
Iteração #: 54
Epoch 1/1
12475/12475 [==============================] - 29s 2ms/step - loss: 6.5271e-04 - accuracy: 0.9998
Iteração #: 55
Epoch 1/1
12475/12475 [==============================] - 28s 2ms/step - loss: 6.2840e-04 - accuracy: 0.9998
Iteração #: 56
Epoch 1/1
12475/12475 [===

In [79]:
# Avalia o modelo
loss, accuracy = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE, verbose=0)
print(model.metrics_names)
print("Accuracy: ", accuracy)
print("Loss: ", loss)

['loss', 'accuracy']
Accuracy:  0.9996774792671204
Loss:  0.002805376832015006


In [80]:
def completar_frase_palavra(frase_inicial):
    palavras = [w for w in word_tokenize(frase_inicial) if w in words]
    for i in range(NUM_PREDS):
        Xtest = np.zeros((1, SEQLEN_WORD, nb_words))
        for j, w in enumerate(palavras[-5:]):
            Xtest[0, j, word2index[w]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2word[np.argmax(pred)]

        # Exibe o próximo caracter previsto
        print(ypred, end=" ")
        palavras.append(ypred)

### Resultados treinamento palavra

In [81]:
frases = ["não esta sendo fácil é difícil ",
         "porque eu te amo e não vivo ",
         "sofrendo querendo seus beijos ",
         "se a gente conseguir ficar ",
         "apagar as lembranças eu só penso ",
         "meu coração se sente cada vez "]

for index, frase in enumerate(frases):
    print("Frase %i: %s" % (index, frase), end="")
    completar_frase_palavra(frase)
    print("\n")

Frase 0: não esta sendo fácil é difícil não chorar na vejo y fotos no deixa cantado no é mais de do sua rio de outro eu aldeia tejo de de a ele a encantar se vai herança clima mundo a me é amor e que quando já o mãos da da ao pra vezes e da assim até vai gente a a e tão quando tão tão o era voltar seu seu um juntos mundo corpo bye perdão neste de é é com você ô ô fonte do gente é a dança é já bateu e olha eu pessoal deus vou você mamãe cara amor e penso é falar o que que não fé é é é é é preciso preciso ô lugar quiser que mar tudo pra um que o não um seu só eu inventar perfeito favor do a bm7 oh ta de y bm9 rola de la tiempo um que la la teu tanto el libre la tanto la laralay huella y que el bayano apodo el de recitado laralay laralay sí saúde fazebu a a hoy agonizando los essa é é a corazón retorno a bateu deste amor ai se para arrancar na se bem bem assim da que no rolar o da 

Frase 1: porque eu te amo e não vivo sem você só eu pares você mas mas acabei esa se a vida de dançar ela c